# Synthetic Device Entities

This notebook is for creating synthetic entites based on the previously seeded devices for each area.

## Model Client

In [3]:
import openai

from home_assistant_datasets.secrets import get_secret
from home_assistant_datasets import model_client

MODEL_ID = "gpt-3.5-turbo-0125"

openai = openai.OpenAI(api_key=get_secret("openai_key"))
model = model_client.ModelClient(openai, MODEL_ID)

## Synthetic Devices

In [51]:
import pathlib
import yaml

DATASET_DIR = pathlib.Path("../datasets/")
SEEDS_DIR = DATASET_DIR / "seeds"
TYPES_FILE = SEEDS_DIR / "types.yaml"
DEVICE_ENTITIES_FILE = SEEDS_DIR / "device-entities.yaml"

with open(TYPES_FILE) as f:
    seeds = yaml.load(f.read(), Loader=yaml.Loader)

platforms = seeds["platforms"]
sensor_device_classes = seeds["sensor_device_classes"]
binary_sensor_device_classes = seeds["binary_sensor_device_classes"]

with open(DEVICE_ENTITIES_FILE) as f:
    seed_device_entities = list(yaml.load_all(f.read(), Loader=yaml.Loader))

In [38]:
DEVICE_ENTITY_PROMPT_FORMAT = """
Input:
{home}
Output:
{device_entities}
"""

item = []
for seed in seed_device_entities:
    print(seed)
    home = yaml.dump(seed["home"], sort_keys=False)
    entities = yaml.dump(seed["device_entities"], sort_keys=False)
    item.append(DEVICE_ENTITY_PROMPT_FORMAT.format(home=home, device_entities=entities))

seed_device_entity_prompt = "\n".join(item)


{'home': [{'name': 'Coastal Bungalow', 'thoughts': ['The bungalow in a seaside village may be a vacation home or a tranquil retreat.', 'The cozy living room with a fireplace hints at a cozy atmosphere, and there may be smart climate control for comfort.', 'The outdoor shower is convenient for beach days, so there may be smart water heating systems.'], 'desc': 'Bungalow in a seaside village in Norway', 'area_devices': {'Living Room': ['light', 'climate_control'], 'Bedrooms': ['light'], 'Porch': ['light'], 'Outdoor Shower': ['water_heater']}, 'other_devices': ['smartphone', 'laptop']}], 'device_entities': [{'Living Room': [{'name': 'light', 'entities': ['light.living_room']}, {'name': 'climate_control', 'entities': ['climage.living_room', 'sensor.living_room_temperature', 'sensor.living_room_humidity']}]}, {'Bedrooms': [{'name': 'light', 'entities': ['light.bedroom']}]}, {'Porch': [{'name': 'light', 'entities': ['light.porch']}]}, {'Outdoor Shower': [{'name': 'water_heater', 'entities': 

In [52]:
DEVICE_TYPE_PROMPT = f"""
You are generating synthetic data to used to train models for Home Assistant
and used to evaluate things like generating a summary, performing home automation
actions, or for generating other synthetic data.

You use your knowledge about the world to generate details about devices taht
can be used for synthetic smart home automation data. For example, an apartment
may have a smart thermostat, a house may have a smart garage door opener or 
smart lock and camera, and all houses may have a smart light or weather feed
air quality, or a smart speaker or television. The needs of a home owner
may vary if they are a single person or family, or where in the world they
live. For example, a person living in a high rise may not have a backyard. It
helps to think step by step when generating the data.

A device in Home Assistant represents a physical or virtual object, but really
it is rare to interact with the device itself. Instead, you interact with the
device through the entities that represent the device. For example a smart
light bulb may have an entity that represents the light bulb and another
entity that represents the light switch. Another example is a smart thermostat
may be represented as a climate entity and a sensor entity that represents
the temperature, and another sensor that represents the humited. A smart dishwasher
may be represented as a set of sensor entities, a select entity for the clean cycle
mode, and switch entities.

The following entity types are supported:
{platforms}

Below are example inputs and outputs for generating device entities for a home.

{seed_device_entity_prompt}
"""

In [55]:
from tqdm.auto import tqdm

AREA_DEVICES_YAML = DATASET_DIR / "area-devices.yaml"
DEVICE_ENTITIES_YAML = DATASET_DIR / "device-entities.yaml"

with open(AREA_DEVICES_YAML) as f:
    content = f.read()

data = yaml.safe_load(content)

skipped = 0
with open(DEVICE_ENTITIES_YAML, "w") as device_output:
    with tqdm(total=len(data)) as pbar:
        for home in data:
            response_obj = None
            for i in range(3):
                prompt = DEVICE_ENTITY_PROMPT_FORMAT.format(home=home, device_entities="")
                response = model.complete(DEVICE_TYPE_PROMPT, prompt)
                try:
                    response_obj = yaml.safe_load(response)
                except yaml.YAMLError:
                    continue
            if response_obj is not None:
                updated_home = home.copy()
                updated_home.update({"device_entities": response_obj})
                device_output.write(yaml.dump(updated_home, explicit_start=True, sort_keys=False))
            pbar.set_description(f"Skipped {skipped}")
            pbar.update(1)

Skipped 0:  20%|██        | 1/5 [00:08<00:35,  8.92s/it]

[{'Living Space': [{'name': 'light', 'entities': ['light.living_space']}, {'name': 'speaker', 'entities': ['media_player.living_space']}]}, {'Kitchen': [{'name': 'light', 'entities': ['light.kitchen']}]}, {'Bathroom': [{'name': 'light', 'entities': ['light.bathroom']}]}, {'Terrace': [{'name': 'light', 'entities': ['light.terrace']}, {'name': 'speaker', 'entities': ['media_player.terrace', 'number.terrace_speaker_balance', 'number.terrace_speaker_treble', 'number.terrace_speaker_mid', 'number.terrace_speaker_bass', 'switch.terrace_speaker_surround_enabled', 'switch.terrace_speech_enhancement']}]}, {'Other': [{'name': 'laptop', 'entities': ['device_tracker.laptop', 'sensor.laptop_battery_level']}, {'name': 'smartphone', 'entities': ['device_tracker.android', 'sensor.android_battery_level', 'sensor.android_battery_temperature', 'sensor.android_battery_voltage']}, {'name': 'tablet', 'entities': ['device_tracker.tablet']}, {'name': 'smartwatch', 'entities': ['device_tracker.watch', 'sensor.

Skipped 0:  40%|████      | 2/5 [00:23<00:36, 12.03s/it]

[{'Bedrooms': [{'name': 'light', 'entities': ['light.bedroom1', 'light.bedroom2', 'light.bedroom3', 'light.bedroom4', 'light.bedroom5', 'light.bedroom6', 'light.bedroom7']}, {'name': 'smart_tv', 'entities': ['media_player.bedroom1_tv', 'remote.bedroom1_tv', 'binary_sensor.bedroom1_tv_headphones_connected', 'media_player.bedroom2_tv', 'remote.bedroom2_tv', 'binary_sensor.bedroom2_tv_headphones_connected', 'media_player.bedroom3_tv', 'remote.bedroom3_tv', 'binary_sensor.bedroom3_tv_headphones_connected', 'media_player.bedroom4_tv', 'remote.bedroom4_tv', 'binary_sensor.bedroom4_tv_headphones_connected', 'media_player.bedroom5_tv', 'remote.bedroom5_tv', 'binary_sensor.bedroom5_tv_headphones_connected', 'media_player.bedroom6_tv', 'remote.bedroom6_tv', 'binary_sensor.bedroom6_tv_headphones_connected', 'media_player.bedroom7_tv', 'remote.bedroom7_tv', 'binary_sensor.bedroom7_tv_headphones_connected']}]}, {'Living Room': [{'name': 'light', 'entities': ['light.living_room']}, {'name': 'smart_t

Skipped 0:  60%|██████    | 3/5 [00:28<00:17,  8.88s/it]

[{'Living Room': [{'name': 'light', 'entities': ['light.living_room']}, {'name': 'speaker', 'entities': ['media_player.living_room']}]}, {'Kitchen': [{'name': 'light', 'entities': ['light.kitchen']}]}, {'Bedrooms': [{'name': 'light', 'entities': ['light.bedroom']}]}, {'Garden': [{'name': 'light', 'entities': ['light.garden']}, {'name': 'speaker', 'entities': ['media_player.garden']}, {'name': 'smartphone', 'entities': ['device_tracker.android']}]}]


Skipped 0:  80%|████████  | 4/5 [00:35<00:08,  8.07s/it]

[{'Living Room': [{'name': 'light', 'entities': ['light.living_room']}, {'name': 'smart_tv', 'entities': ['media_player.living_room_tv', 'remote.living_room_tv', 'binary-sensor.living_room_tv_headphones_connected']}]}, {'Kitchen': [{'name': 'light', 'entities': ['light.kitchen']}]}, {'Bedrooms': [{'name': 'light', 'entities': ['light.bedroom']}]}, {'Balcony': [{'name': 'light', 'entities': ['light.balcony']}]}, {'Other': [{'name': 'laptop', 'entities': ['device_tracker.laptop']}, {'name': 'smartphone 1', 'entities': ['device_tracker.smartphone_1']}, {'name': 'smartphone 2', 'entities': ['device_tracker.smartphone_2']}]}]


Skipped 0: 100%|██████████| 5/5 [00:41<00:00,  8.26s/it]

[{'Living Room': [{'name': 'light', 'entities': ['light.living_room']}, {'name': 'climate_control', 'entities': ['climate.living_room', 'sensor.living_room_temperature', 'sensor.living_room_humidity']}]}, {'Bedrooms': [{'name': 'light', 'entities': ['light.bedroom']}, {'name': 'climate_control', 'entities': ['climate.bedroom', 'sensor.bedroom_temperature', 'sensor.bedroom_humidity']}]}, {'Porch': [{'name': 'light', 'entities': ['light.porch']}]}, {'Outdoor Shower': [{'name': 'water_heater', 'entities': ['water_heater.outdoor_shower', 'sensor.outdoor_shower_temperature']}]}, {'Other': [{'name': 'laptop', 'entities': ['device_tracker.laptop']}, {'name': 'smartphone', 'entities': ['device_tracker.android']}]}]
